In [1]:
import torch

In [3]:
import torch.nn as nn

In [ ]:
class Erode(nn.Module):
    def forward(self, mask: torch.Tensor) -> torch.Tensor:
            

In [34]:
mask = torch.zeros((1, 10, 10), dtype=bool)
mask[:, 2:7, 1:8] = True

In [28]:
from math import sqrt
umask = nn.Unfold((3, 3))(mask)


In [29]:
umask.shape

torch.Size([1, 9, 64])

In [41]:
mask.unfold(1, 3, 1).unfold(2, 3, 1).reshape(64, 9)

AttributeError: 'torch.return_types.max' object has no attribute 'view'

torch.Size([1, 1, 10, 10])